In [1]:
import numpy as np
import astropy
from astropy.io import fits
from matplotlib import pyplot
import pandas as pd
import plotting as my_plot

In [2]:
def h2cum(arr):
    res = 1. * arr[::-1]
    for i in range(1, len(arr)):
        res[i] += res[i - 1]
    return res[::-1]

In [3]:
my_plot.setup_figure_pars()
save_plots = 1
plot_assoc_prob = 1
weight_str = 'unweighted' # 'unweighted'
alg0 = 'LR' # 'RF', 'BDT', 'LL', 'NN'
algs = ['RF', 'BDT', 'LR', 'NN']

prop_cycle = pyplot.rcParams['axes.prop_cycle']
cs = prop_cycle.by_key()['color']



high_lat = True
#cat = '4FGL' # '3FGL', '4FGL'
fn3 = '../PS_comparison/gll_psc_v16.fit' # the last release of 3FGL
fn4 = '../PS_comparison/gll_psc_v22.fit' # 4FGL release from 27 May 2020
fn4dr2 = '../PS_comparison/gll_psc_v24.fit' # 4FGL DR2
fn3pcat = '../ML_3FGL_paper/data/3fgl_unassoc_vs_4fgl_assoc.csv'

folder = '../ML_3FGL_paper/data/python files/catas'
if 1:
    fn3fgl_assoc = '%s/3FGL_assoc_catalog_%s.csv' % (folder, weight_str)
    fn3fgl_unassoc = '%s/3FGL_unassoc_catalog_%s.csv' % (folder, weight_str)
    fn4fgl_assoc = '%s/4FGL_assoc_catalog_%s.csv' % (folder, weight_str)
    fn4fgl_unassoc = '%s/4FGL_unassoc_catalog_%s.csv' % (folder, weight_str)
else:
    fn3fgl_assoc = '%s/3fgl_all_assoc_catalog_%s.csv' % (folder, weight_str)
    fn3fgl_unassoc = '%s/3fgl_all_unassoc_catalog_%s.csv' % (folder, weight_str)
    fn4fgl_assoc = '%s/4fgl_all_assoc_catalog_%s.csv' % (folder, weight_str)
    fn4fgl_unassoc = '%s/4fgl_all_unassoc_catalog_%s.csv' % (folder, weight_str)
    
pop = 'AGN' #'PSR', 'AGN'
quantity = 'Flux1000'
if plot_assoc_prob:
    figfn = 'plots/logN_logS_%s_%s.pdf' % (pop, weight_str)
    figfn_diff = 'plots/logN_logS_diff_%s_%s.pdf' % (pop, weight_str)
else:
    figfn = 'plots/logN_logS_%s.pdf' % (pop)
    figfn_diff = 'plots/logN_logS_diff_%s.pdf' % (pop)
    
hdu3 = fits.open(fn3)
hdu4 = fits.open(fn4)
#pcat_3fgl_vs_4fgl = pd.read_csv(fn3pcat, index_col='Source_Name_3FGL')
pcat_3fgl_assoc = pd.read_csv(fn3fgl_assoc, index_col='Source_Name_3FGL')
pcat_3fgl_unassoc = pd.read_csv(fn3fgl_unassoc, index_col='Source_Name_3FGL')
pcat_4fgl_assoc = pd.read_csv(fn4fgl_assoc, index_col='Source_Name_4FGL')
pcat_4fgl_unassoc = pd.read_csv(fn4fgl_unassoc, index_col='Source_Name_4FGL')
#pcat = np.loadtxt(fn3pcat, delimiter=',', dtype=str)

#pcat_3fgl_unassoc = pcat_3fgl_unassoc
#pcat_4fgl_unassoc = pcat_4fgl_unassoc

index = np.array(hdu3[1].data.field('Source_Name'), dtype=str)
data = {quantity:np.array(hdu3[1].data.field(quantity), dtype=float)}
df_3fgl = pd.DataFrame(data=data, index=index)
df_3fgl.index = [st.strip() for st in df_3fgl.index]
index = hdu4[1].data.field('Source_Name')
data = {quantity:np.array(hdu4[1].data.field(quantity), dtype=float)}
df_4fgl = pd.DataFrame(index=index, data=data)
df_4fgl.index = [st.strip() for st in df_4fgl.index]

FileNotFoundError: [Errno 2] File ../ML_3FGL_paper/data/python files/catas/3FGL_assoc_catalog_unweighted.csv does not exist: '../ML_3FGL_paper/data/python files/catas/3FGL_assoc_catalog_unweighted.csv'

In [4]:
hdu = fits.open(fn4dr2)

In [15]:
np.sum(hdu[1].data.field('CLASS1') == 'PSR')

235

In [ ]:
# flux or another value for associated and unassociated sources
if quantity == 'index':
    value_3fgl = hdu3[1].data.field('Spectral_Index')
    value_4fgl = hdu4[1].data.field('PL_Index')
elif quantity == 'Flux1000':
    df_3fgl['value'] = np.log10(np.array(df_3fgl[quantity]))
    df_4fgl['value'] = np.log10(np.array(df_4fgl[quantity]))
    if 0:
        index = pcat_3fgl_unassoc.index
        value_pcat_3fgl = np.array(df_3fgl.loc[index, quantity])
        value_pcat_3fgl = np.log10(value_pcat_3fgl)
        index = pcat_4fgl_unassoc.index
        value_pcat_4fgl = np.array(df_4fgl.loc[index, quantity])
        value_pcat_4fgl = np.log10(value_pcat_4fgl)


In [ ]:
psclass3 = np.array(list(hdu3[1].data.field('CLASS1')))
print(np.sum(psclass3 == 'PSR') + np.sum(psclass3 == 'psr'))
psclass4 = np.array(list(hdu4[1].data.field('CLASS1')))
print(np.sum(psclass4 == 'PSR') + np.sum(psclass4 == 'psr'))

In [ ]:
class_mask_3fgl_assoc = (pcat_3fgl_assoc['Category_3FGL'] == pop)
source_names_3fgl = [name for name in pcat_3fgl_assoc.index if class_mask_3fgl_assoc[name]]
# temporary hack:
if 0:
    labels = {}
    labels['AGN'] = ["agn", "bcu", "bll", "css", "fsrq", "nlsy1", "rdg", "sey", "ssrq"]
    labels['PSR'] = ["PSR", "psr",]
    labels['other'] = ["bin", "hmb", "nov", "PSR", "psr", "pwn",  "snr", "spp", "glc", "sfr"]
    psclass4 = list(hdu4[1].data.field('CLASS1'))

    n4 = len(df_4fgl.index)
    source_names_4fgl = [df_4fgl.index[i] for i in range(n4) if psclass4[i] in labels[pop]]
else:
    class_mask_4fgl_assoc = (pcat_4fgl_assoc['Category_4FGL'] == pop)
    source_names_4fgl = [name for name in pcat_4fgl_assoc.index if class_mask_4fgl_assoc[name]]

minv = np.min(df_3fgl.loc[source_names_3fgl, 'value'])
maxv = np.max(df_3fgl.loc[source_names_3fgl, 'value'])
bins = np.linspace(minv,maxv, 20)
binsc = (bins[1:] + bins[:-1])/2
print(bins)

In [ ]:

# correction for other sources
other_names_3fgl = [name for name in pcat_3fgl_assoc.index if 
                        pcat_3fgl_assoc.loc[name, 'Category_3FGL'] == 'OTHER']
other_values_3fgl = df_3fgl.loc[other_names_3fgl, 'value']

hist_all_assoc_3fgl = np.histogram(df_3fgl.loc[pcat_3fgl_assoc.index, 'value'], bins=bins)

#hist_all_assoc_3fgl = np.histogram(df_3fgl.loc[pcat_3fgl_assoc.index, 'value'], bins=bins)
hist_all_unassoc_3fgl = np.histogram(df_3fgl.loc[pcat_3fgl_unassoc.index, 'value'], bins=bins)
other_3fgl_corr = {}
other_3fgl_assoc_corr = {}
for alg in algs:
    label = '%s_%s' % (pop, alg)
    probs = pcat_3fgl_assoc.loc[other_names_3fgl, label]
    hist_other_3fgl = np.histogram(other_values_3fgl, bins=bins, weights=probs)
    other_3fgl_corr[alg] = hist_other_3fgl[0]/(hist_all_assoc_3fgl[0] + 0.00001) * hist_all_unassoc_3fgl[0]
    #other_3fgl_corr = 0.
    other_3fgl_assoc_corr[alg] = hist_other_3fgl[0]

# 4FGL
other_names_4fgl = [name for name in pcat_4fgl_assoc.index if 
                        pcat_4fgl_assoc.loc[name, 'Category_4FGL'] == 'OTHER']
other_values_4fgl = df_4fgl.loc[other_names_4fgl, 'value']

hist_all_assoc_4fgl = np.histogram(df_4fgl.loc[pcat_4fgl_assoc.index, 'value'], bins=bins)
hist_all_unassoc_4fgl = np.histogram(df_4fgl.loc[pcat_4fgl_unassoc.index, 'value'], bins=bins)
other_4fgl_corr = {}
other_4fgl_assoc_corr = {}
for alg in algs:
    label = '%s_%s' % (pop, alg)
    probs = pcat_4fgl_assoc.loc[other_names_4fgl, label]
    hist_other_4fgl = np.histogram(other_values_4fgl, bins=bins, weights=probs)
    other_4fgl_corr[alg] = hist_other_4fgl[0]/(hist_all_assoc_4fgl[0] + 0.00001) * hist_all_unassoc_4fgl[0]
    other_4fgl_assoc_corr[alg] = hist_other_4fgl[0]


In [ ]:
# probabilities and masks (for associated sources)
unassoc_values_3fgl = df_3fgl.loc[pcat_3fgl_unassoc.index, 'value']
assoc_values_3fgl = df_3fgl.loc[pcat_3fgl_assoc.index, 'value']
unassoc_values_4fgl = df_4fgl.loc[pcat_4fgl_unassoc.index, 'value']
assoc_values_4fgl = df_4fgl.loc[pcat_4fgl_assoc.index, 'value']
hpcat_3fgl = {}
hpcat_4fgl = {}
hpcat_3fgl_assoc = {}
hpcat_4fgl_assoc = {}
prob_3fgl_unassoc = {}
prob_3fgl_assoc = {}
prob_4fgl_unassoc = {}
prob_4fgl_assoc = {}
for alg in algs:
    label = '%s_%s' % (pop, alg)
    prob_3fgl_unassoc[alg] = np.array(pcat_3fgl_unassoc[label]) # prob
    prob_4fgl_unassoc[alg] = np.array(pcat_4fgl_unassoc[label]) # prob4
    prob_3fgl_assoc[alg] = np.array(pcat_3fgl_assoc[label]) # prob
    prob_4fgl_assoc[alg] = np.array(pcat_4fgl_assoc[label]) # prob4
    hpcat_3fgl[alg] = np.histogram(unassoc_values_3fgl, bins=bins, 
                                   weights=prob_3fgl_unassoc[alg])[0]
    hpcat_4fgl[alg] = np.histogram(unassoc_values_4fgl, bins=bins, 
                                   weights=prob_4fgl_unassoc[alg])[0]
    hpcat_3fgl_assoc[alg] = np.histogram(assoc_values_3fgl, bins=bins, 
                                   weights=prob_3fgl_assoc[alg])[0]
    hpcat_4fgl_assoc[alg] = np.histogram(assoc_values_4fgl, bins=bins, 
                                   weights=prob_4fgl_assoc[alg])[0]

In [ ]:
print(other_3fgl_corr['LR']/(hpcat_3fgl['LR'] + 0.0001))

In [ ]:
print(hpcat_3fgl_assoc['LR'])


In [ ]:
print(hpcat_3fgl['LR'])

In [ ]:
if 1:
    pyplot.figure()
    h3 = pyplot.hist(df_3fgl.loc[source_names_3fgl, 'value'], 
                         bins=bins, color='b', alpha=0.3)
    h4 = pyplot.hist(df_4fgl.loc[source_names_4fgl, 'value'], 
                         bins=bins, color='g', alpha=0.3)
    hpcat_3fgl0 = pyplot.hist(unassoc_values_3fgl, bins=bins, color='r', 
                                alpha=0.3, weights=prob_3fgl_unassoc[alg0])
    hpcat_4fgl0 = pyplot.hist(unassoc_values_4fgl, bins=bins, color='k', 
                                 alpha=0.3, weights=prob_4fgl_unassoc[alg0])

In [ ]:
def min_max_vs(dct, corr={}):
    vals = np.array([dct[key] - corr.get(key, 0.) for key in dct.keys()])
    return np.min(vals, axis=0), np.max(vals, axis=0)

In [ ]:
hpcat_3fgl0[0]/(h3[0] + 0.0000001)

In [ ]:
if pop == 'AGN' or 1:
    pyplot.rcParams['figure.subplot.left'] = 0.18
    pyplot.rcParams['figure.subplot.right'] = 0.98
elif pop == 'PSR':
    pyplot.rcParams['figure.subplot.left'] = 0.15
    pyplot.rcParams['figure.subplot.right'] = 0.95


#plot = pyplot.semilogy 
plot = pyplot.plot
    
xs = binsc
pyplot.figure()
plot(xs, h2cum(h3[0]), c=cs[0], ls='-', label='3FGL')
pcat_hist_3fgl = hpcat_3fgl[alg0] - other_3fgl_corr[alg0]
pcat_hist_3fgl_assoc = hpcat_3fgl_assoc[alg0] - other_3fgl_assoc_corr[alg0]

plot(xs, h2cum(h3[0] + pcat_hist_3fgl), c=cs[2], ls='--', label='3FGL + unassoc. prob.')
if plot_assoc_prob:
    plot(xs, h2cum(pcat_hist_3fgl_assoc), c=cs[4], ls='--', label='3FGL assoc. prob.')
    minvs, maxvs = min_max_vs(hpcat_3fgl_assoc, corr=other_3fgl_assoc_corr)
    pyplot.fill_between(xs, h2cum(minvs), h2cum(maxvs), color=cs[4], alpha=0.5)
    
if 0:
    pcat_hist = hpcat_3fgl[alg0]
    plot(xs, h2cum(h3[0] + pcat_hist), c=cs[5], ls=':', label='3FGL + unassoc. prob. (uncorr)')
minvs, maxvs = min_max_vs(hpcat_3fgl, corr=other_3fgl_corr)
pyplot.fill_between(xs, h2cum(h3[0] + minvs), h2cum(h3[0] + maxvs), color=cs[2], alpha=0.5)


plot(xs, h2cum(h4[0]), c=cs[1], ls='-.', label='4FGL')
pcat_hist_4fgl = hpcat_4fgl[alg0] - other_4fgl_corr[alg0]
pcat_hist_4fgl_assoc = hpcat_4fgl_assoc[alg0] - other_4fgl_assoc_corr[alg0]

plot(xs, h2cum(h4[0] + pcat_hist_4fgl), c=cs[3], ls=':', label='4FGL + unassoc. prob.')
if plot_assoc_prob:
    plot(xs, h2cum(pcat_hist_4fgl_assoc), c=cs[7], ls=':', label='4FGL assoc. prob.')
    minvs, maxvs = min_max_vs(hpcat_4fgl_assoc, corr=other_4fgl_assoc_corr)
    pyplot.fill_between(xs, h2cum(minvs), h2cum(maxvs), color=cs[7], alpha=0.5)

minvs, maxvs = min_max_vs(hpcat_4fgl, corr=other_4fgl_corr)
pyplot.fill_between(xs, h2cum(h4[0] + minvs), h2cum(h4[0] + maxvs), color=cs[3], alpha=0.5)
pyplot.xlabel(r'$\log_{10} F_{>1 GeV}\ [\rm cm^{-2}s^{-1}]$')
pyplot.ylabel('Number of sources')
pyplot.legend()
title = pop
#if plot_assoc_prob:
#    title += ' (%s)' % weight_str
pyplot.title(title)
if save_plots:
    print('Save figure to file:', figfn)
    pyplot.savefig(figfn)

In [ ]:
# checks
print('All unassociated surces')
print(hist_all_unassoc_3fgl[0])
print('All associated surces')
print(hist_all_assoc_3fgl[0])
print('Associated surces of %s class' % pop)
print(h3[0])
print('Other associated surces')
print(hist_other_3fgl[0])
print('Estimated number of %s sources from probabilities' % pop)
print(pcat_hist_3fgl_assoc)
#print(other_3fgl_corr['LR']/(hpcat_3fgl['LR'] + 0.0001))


In [ ]:
print(other_3fgl_corr[alg0] / (h3[0] + hpcat_3fgl[alg0] + 0.0001))

In [ ]:
eps = 1.e-10
pyplot.rcParams['figure.subplot.left'] = 0.18
pyplot.rcParams['figure.subplot.right'] = 0.98

pyplot.figure()
pyplot.plot(binsc, pcat_hist_3fgl / (h3[0] + eps), c=cs[0], ls='-', label='3FGL')
#pyplot.plot(binsc, hpcat_3fgl0[0], c=cs[2], ls='--', label='3FGL unassoc. prob.')
pyplot.plot(binsc, np.sign(h4[0]) * pcat_hist_4fgl / (h4[0] + eps), c=cs[1], ls='-.', label='4FGL')
#pyplot.plot(binsc, hpcat_4fgl0[0], c=cs[3], ls=':', label='4FGL unassoc. prob.')

pyplot.legend()

pyplot.xlabel(r'$\log_{10} F_{>1 GeV}\ [\rm cm^{-2}s^{-1}]$')
pyplot.ylabel('unassoc. / assoc. fraction')
pyplot.legend()
title = pop
#if plot_assoc_prob:
#    title += ' (%s)' % weight_str
pyplot.title(title)

if save_plots:
    print('Save figure to file:', figfn_diff)
    pyplot.savefig(figfn_diff)
    command = 'cp %s ../ML_3FGL_paper/plots' % figfn_diff
    os.system(command)

In [ ]:
if 0:
    col_names3 = [hdu3[1].header[key] for key in hdu3[1].header.keys() if key.startswith('TTYPE')]
    col_names4 = [hdu4[1].header[key] for key in hdu4[1].header.keys() if key.startswith('TTYPE')]
    #print(col_names3)
    #print(col_names4)
    
    labels = {}
    labels['AGN'] = ["agn", "bcu", "bll", "css", "fsrq", "nlsy1", "rdg", "sey", "ssrq"]
    labels['PSR'] = ["PSR", "psr",]
    labels['other'] = ["bin", "hmb", "nov", "PSR", "psr", "pwn",  "snr", "spp", "glc", "sfr"]
    psclass4 = list(hdu4[1].data.field('CLASS1'))


In [ ]:
if 0:
    psclass3 = list(hdu3[1].data.field('CLASS1'))
    psclass4 = list(hdu4[1].data.field('CLASS1'))

    n3 = len(psclass3)
    n4 = len(psclass4)

    inds3 = [i for i in range(n3) if psclass3[i] in pop_key]
    inds4 = [i for i in range(n4) if psclass4[i] in pop_key]

    
    print(len(value_pcat_3fgl))
    print(len(prob), sum(prob))